# Design of Experiments with and for Machine Learning

In this notebook, we will implement **Bayesian optimization** techniques as adative sampling methodes for experimental design. 

At the start, we will implement a *Gaussian process* (GP) for regression problems.

Subsequently, we will use GPs as surrogate probabilistic models to implement the acquisition functions
- *probability improvement* (PI),
- *expected improvement* (EI),
- and *upper confidence bound* (UBC).

Finally, we will employ the implemented acquisition functions on synthethic data sets to study their behaviors.

### **Table of Contents**
1. [Gaussian Processes for Regression](#gpr)
2. [Acquisition Functions](#af)
3. [Bayesian Optimization](#bo)

In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interactive, FloatSlider, IntSlider, Dropdown
from sklearn.datasets import make_blobs
from sklearn.metrics.pairwise import pairwise_kernels
from functools import partial

### **1. Gaussian Processes for Regression** <a class="anchor" id="gpr"></a>

**Kernel functions** are central components of Gaussian processes. A kernel function $k: \mathcal{X} \rightarrow \mathbb{R}$ is typically chosen to quantify a kind of similarity between two arbitrary instances $\mathbf{x}_n$ and $\mathbf{x}_m$. This similary measurement corresponds to a dot product of two instances in a transformed Hilbert space:
$$
k(\mathbf{x}, \mathbf{x}^\prime) = \boldsymbol{\phi}(\mathbf{x})^\mathrm{T} \boldsymbol{\phi}(\mathbf{x}^\prime),
$$
where $\boldsymbol{\phi}: \mathcal{X} \rightarrow \mathcal{H}$ is the feature map of the corresponding kernel function. Accordingly, we can use this property to prove whether a function defines a valid kernel. As a simple example, consider a kernel function given by
$$
k(\mathbf{x}, \mathbf{x}^\prime) = (\mathbf{x}^\mathrm{T}\mathbf{x}^\prime)^2.
$$
If we take the particular case of a two-dimensional instance space $\mathcal{X} = \mathbb{R}^2$ we can expand out the terms and thereby identify the following nonlinear feature mapping:
$$
(x[0] * x'[0] + x[1] * x'[1])^2
$$
There are several kernel functions, which can be used in combination with a Gaussian process regression model. 
For simplicity, we will use the [`pairwise_kernels`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_kernels.html) as part of the scikit-learn package. In the following, we see different functions $f$ drawn from the Gaussian prior $\mathcal{N}(\mathbf{f} \mid \mathbf{0}, \mathbf{K})$, where the Gram matrix $\mathbf{K} \in \mathbb{R}^{N \times N}$ is computed through the radial basis function (RBF) kernel, also known as Gaussian kernel, defined as:
$$
k(x, x') = exp(\frac{-(x-x')^2}{2*\sigma^2})
$$


In [2]:
from sklearn.metrics.pairwise import rbf_kernel

def visualize_prior_functions(gamma):
    # Create samples (instances).
    x_axis = np.linspace(-1, 1, 100)
    X = x_axis.reshape(-1, 1)

    # Compute Gram matrix `K` using RBF kernel
    # with `gamma`.
    K = 1.0 *  rbf_kernel(X, gamma = gamma)

    # Draw 5 function values `f` ~ N(0, K).
    functions = []
    for i in range(5):
        functions[i] = np.random.normal(loc = 0.0, scale = K, size = len(x_axis))

        # Plot drawn samples `f`.
        plt.plot(x_axis, functions[i], label=f"function {i}")
    plt.legend()
    plt.show()
    
interactive(
    visualize_prior_functions, 
    gamma=FloatSlider(value=1, min=0.01, max=10), 
)

interactive(children=(FloatSlider(value=1.0, description='gamma', max=10.0, min=0.01), Output()), _dom_classes…

The key results defining a Gaussian process are given through the mean and variance of the conditional distribution $p(y_{N+1} \mid \mathcal{D}_N)$. The corresponding formulas are given in the following:

TODO

With this knowledge, we implement the class [`GaussianProcessRegression`](../e2ml/models/_gaussian_process_regression.py) in the [`e2ml.models`](../e2ml/models) subpackage.
Once, the implementation has been completed, we check its validity on a simple one-dimensional artificial data set through visual inspection. 

In [3]:
from e2ml.models import GaussianProcessRegression

def objective_function(x, noise):
    noise = np.random.normal(loc=0, scale=noise, size=len(x)) if noise > 0 else 0
    return (np.sin(2 * np.pi * x)**6.0 * x) + noise

def generate_data(noise, train_ratio):
    x_axis = np.linspace(-1, 1, 100)
    X = x_axis.reshape(-1, 1)
    n_train_samples = int(train_ratio * len(X))
    train_idx = np.random.choice(np.arange(len(X)), replace=False, size=n_train_samples)
    X_train = X[train_idx]
    y_train = objective_function(X_train.ravel(), noise=noise)
    f = objective_function(x_axis, noise=0)
    return x_axis, X, f, X_train, y_train

def visualize_gpr_predictions(noise=0.1, beta=0.1, gamma=1, train_ratio=0.5):
    # Create samples (instances) and targets.
    x_axis, X, f, X_train, y_train = generate_data(noise, train_ratio)
    
    # Create Gaussian process.
    metrics_dict = {'gamma': gamma, 'metric': 'rbf'}
    gpr = GaussianProcessRegression(beta=beta, metrics_dict=metrics_dict)
    
    # Fit Gaussian process on training data.
    gpr.fit(X_train, y=y_train)

    # Predict `means` and `stds`.
    means, stds = gpr.predict(X, True)
    
    # Visualize targets and predictions.
    plt.figure(figsize=(16, 9))
    plt.scatter(X_train.ravel(), y_train, s=20, c='k')
    plt.plot(x_axis, f, label="$f(x)$", c='r', ls='--')
    plt.plot(x_axis, means, label="$\mu(x)$", c='b')
    plt.fill_between(x_axis, means-stds, means+stds, alpha=0.1, color='b')
    plt.xlabel('$x$', fontsize=15)
    plt.ylabel('$f(x)$', fontsize=15)
    plt.legend(prop={'size': 10})
    plt.show()
    
interactive(
    visualize_gpr_predictions, 
    noise=FloatSlider(value=0.1, min=0.0, max=10),
    beta=FloatSlider(value=0.01, min=0.0, max=2),
    gamma=FloatSlider(value=50, min=0.01, max=100),
    train_ratio=FloatSlider(value=0.5, min=0.1, max=1)
)

interactive(children=(FloatSlider(value=0.1, description='noise', max=10.0), FloatSlider(value=0.01, descripti…

### **2. Acquisition Functions** <a class="anchor" id="af"></a>

Three popular acquisition functions are:
- probability improvement (PI) defined as:
$$
\alpha_{PI}(\bold{x}|\mathcal{D_N}) = \Phi(\frac{\mu(\bold{x}|\mathcal{D_N})-\tau}{\sigma(\bold{x}|\mathcal{D_N})})
$$

- expected improvement (EI) defined as:
$$
\alpha_{EI}(\bold{x}|\mathcal{D_N}) = (\mu(\bold{x}|\mathcal{D_N})-\tau) \Phi(\frac{\mu(\bold{x}|\mathcal{D_N})-\tau}{\sigma(\bold{x}|\mathcal{D_N})}) + \sigma(\bold{x}|\mathcal{D_N}) \phi(\frac{\mu(\bold{x}|\mathcal{D_N})-\tau}{\sigma(\bold{x}|\mathcal{D_N})})
$$

- and upper confidence bound (UCB) defined as:
$$
\alpha_{UCB}(\bold{x}|\mathcal{D_N}) = \mu(\bold{x}|\mathcal{D_N}) + \kappa_N\sigma(\bold{x}|\mathcal{D_N})
$$

#### Question:
2. (a) What is the task of an acquisition function?

   Evaluate best option for next sample input of experiment.
   Exploration and exploitation trade-off.
   
With this knowledge, we implement the functions 

- [`acquisition_pi`](../e2ml/experimentation/_bayesian_optimization.py),
- [`acquisition_ei`](../e2ml/experimentation/_bayesian_optimization.py),
- and [`acquisition_ucb`](../e2ml/experimentation/_bayesian_optimization.py)

in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage.

Once, the implementations have been completed, we check their validity on a simple one-dimensional artificial data set through visual inspection. 

In [5]:
from e2ml.experimentation import acquisition_pi, acquisition_ei, acquisition_ucb

def visualize_acquisition_functions(noise=0.1, beta=0.1, gamma=1, train_ratio=0.1, kappa=1.):
    # Create samples (instances) and targets.
    x_axis, X, f, X_train, y_train = generate_data(noise, train_ratio)
    
    # Create Gaussian process.
    metrics_dict = {'gamma': gamma, 'metric': 'rbf'}
    gpr = GaussianProcessRegression(beta=beta, metrics_dict=metrics_dict)
    
    # Fit Gaussian process on training data.
    gpr.fit(X_train, y_train)

    # Predict `means` and `stds`.
    means, stds = gpr.predict(X, True)
    
    # Determine `tau`.
    tau = 0 # find values better than tau
    
    # Compute PI scores as `pi_scores`.
    pi_scores = acquisition_pi(means, stds, tau)
    
    # Compute EI scores as `ei_scores`.
    ei_scores = acquisition_ei(means, stds, tau)
    
    # Compute UCB scores as `ucb_scores` with `kappa`.
    ucb_scores = acquisition_ucb(means, stds, kappa)
    
    # Visualize targets and predictions.
    plt.figure(figsize=(16, 9))
    plt.scatter(X_train.ravel(), y_train, s=20, c='k')
    plt.plot(x_axis, f, label="$f(x)$", c='r', ls='--')
    plt.plot(x_axis, means, label="$\mu(x)$", c='b')
    plt.plot(x_axis, pi_scores, label='PI', c='g')
    plt.plot(x_axis, ei_scores, label='EI', c='g', ls='--')
    plt.plot(x_axis, ucb_scores, label='UCB', c='g', ls=':')
    plt.fill_between(x_axis, means-stds, means+stds, alpha=0.1, color='b')
    plt.xlabel('$x$', fontsize=15)
    plt.ylabel('$f(x)$', fontsize=15)
    plt.legend(prop={'size': 12})
    plt.show()
    
interactive(
    visualize_acquisition_functions, 
    noise=FloatSlider(value=0.1, min=0.0, max=2),
    beta=FloatSlider(value=0.01, min=0.0, max=2),
    gamma=FloatSlider(value=50, min=0.01, max=100),
    train_ratio=FloatSlider(value=0.05, min=0.05, max=1),
    kappa=FloatSlider(value=1, min=0.0, max=3)
)

interactive(children=(FloatSlider(value=0.1, description='noise', max=2.0), FloatSlider(value=0.01, descriptio…

### **3. Bayesian Optimization** <a class="anchor" id="bo"></a>

Since we have implemented the Gaussian process as surrogate probabilistic model and possible acquisition functions, we can perform Bayesian optimization. For this purpose, we implement the function [`perform_bayesian_optimization`](../e2ml/experimentation/_bayesian_optimization.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage. Once, the implementation has been completed, we test it on a simple one-dimensional artificial data set through visual inspection.

In [ ]:
from e2ml.experimentation import perform_bayesian_optimization

def visualize_bayesian_optimization(gamma=50, noise=0.05, beta=0.05, n_evals=100,
                                    n_random_init=5, acquisition_function='pi'):
    # Generate data and true objective values.
    x_axis = np.linspace(-1, 1, 1000)
    X_cand = x_axis.reshape(-1, 1)
    f = objective_function(x_axis, noise=0)
    
    # Partially initialize objective_function.
    obj_func = partial(objective_function, noise=noise)
    
    # Create Gaussian process model.
    metrics_dict = {'gamma': gamma, 'metric': 'rbf'}
    gpr = GaussianProcessRegression(beta=beta, metrics_dict=metrics_dict)
    
    # Perform Bayesian optimization with given parameters
    # to obtain `X_acquired` and `y_acquired`.
    X_acquired, y_acquired = gpr.perform_bayesian_optimization(X_cand, gpr, acquisition_function, obj_func, n_evals, n_random_init)
    
    # Fit Gaussian process as `gpr` on acquired data.
    gpr.fit(X_acquired, y_acquired)
    
    # Predict `means` and `stds` on `X_cand`.
    means, stds = gpr.predict(X_cand)
    
    # Visualize Bayesian optimization process.
    plt.figure(figsize=(16, 9))
    plt.plot(x_axis, means, label="$\mu(x)$", c='b')
    plt.fill_between(x_axis, means-stds, means+stds, alpha=0.1, color='b')
    plt.plot(x_axis, f, label="$f(x)$", c='r', ls='--')
    plt.scatter(X_acquired.ravel(), y_acquired, s=20, c='k', label='acquired samples')
    for i in range(n_evals):
        plt.text(X_acquired[i], y_acquired[i], str(i+1), color="k", fontsize=12)
    plt.xlabel('$x$', fontsize=15)
    plt.ylabel('$f(x)$', fontsize=15)
    plt.legend(prop={'size': 12})
    plt.title('Bayesian optimization with numbers indicating acquisition order', fontsize=15)
    plt.show()
    
interactive(
    visualize_bayesian_optimization, 
    gamma=FloatSlider(value=50, min=0.01, max=100),
    noise=FloatSlider(value=0.1, min=0.0, max=10),
    beta=FloatSlider(value=0.01, min=0.0, max=2),
    n_evals=IntSlider(value=20, min=1, max=500),
    n_random_init=IntSlider(value=5, min=1, max=500),
    acquisition_function=Dropdown(options=['pi', 'ei', 'ucb'], value='pi')
)